In [ ]:
import Pkg
Pkg.activate("../../Tasks2D")
using Revise

In [ ]:
includet("09/model.jl") # Probabilistic model
includet("09/pf.jl")    # Particle filter

includet("09/wall_follow.jl") # Wall following behavior
includet("09/astar_nav.jl")   # Behavior for navigation via A*

In [ ]:
includet("09/visualize.jl")

In [ ]:
# Load the map (line segments, bounding box)
(_segs, _bb, _) = L.load_env_sparse("/Users/georgematheos/Developer/research/summer2023/tasks2d/data/environments/ambig.json");

In [ ]:
# Parameters for the grid discretization used for planning
ϵ = 0.25
(grid, edges, l_to_g, g_to_l) = line_to_grid(_segs, _bb, ϵ)
w = GridWorlds.boolmatrix_to_grid(grid, (length(edges[1]), length(edges[2])));

In [ ]:
### Ground truth world model parameters ###
PARAMS = (;
    map = _segs, orientation = π/2,
    bounding_box = L.bounding_box(_segs),
    step = (; Δ = 0.25, σ = 0.005 ),
    obs = (; fov = 2π, n_rays = 40,
        sensor_args = (;
            w = 5, s_noise = 0.02,
            outlier = 0.0001, outlier_vol = 100.0,
            zmax = 100.0
)));

In [ ]:
### Particle Filter args ###
_grid_args = (;
    tau = 1., pmin = 1e-6, n_iters = 5,
    init_grid_args = (; k = [9, 9], r = [PARAMS.step.Δ/3, PARAMS.step.Δ/3]),
)
_t0_grid_args = (;
    _grid_args..., tau=5., n_iters=6,
    # TODO: set this up to automatically look at the bounding box
    init_grid_args = (; k=[7, 13], r=[.8, .8]), initial_pos = [2.7, 6.]
)

INFERENCE_PARAMS = (;
    PARAMS...,
    step = (; Δ = 0.25, σ = 0.1),
    obs = (; PARAMS.obs..., sensor_args=(;
        PARAMS.obs.sensor_args...,
        s_noise = 0.02,
    )
)

_resampling_args = (; n_particles=40, n_groups=10, ess_threshold=1+1e-4)

### Particle filter ###
pf = @get_pf(PARAMS_NOISYSTEP, _grid_args, _t0_grid_args, _resampling_args);

In [ ]:
(pf_init, pf_update) = pf;

In [ ]:
GOAL = [1., 4.];

In [ ]:
const initial_controller_state =
    (nothing, initial_wall_follow_state(), nothing)

@gen function controller(st, obs)
    # Unpack controller state
    pf_state, wf_state, prev_action = st
    
    # Update belief state
    if isnothing(prev_action) # First timestep
        pf_state = pf_init(obs)
    else
        pf_state = pf_update(pf_state, prev_action, obs)
    end
    
    # Choose action
    if there_is_ambiguity(pf_state)
        (action, next_wf_state) = wall_follow(wall_following_internal_state, pos)
    else
        action = find_action_using_grid_search(pos, GOAL)
        # Reset the wall following policy
        next_wf_state = initial_wall_follow_state()
    end
    
    # Return action and new controller state
    return (action, (pf_state, next_wf_state, action))
end

controller = GenPOMDPs.Controller(controller, initial_controller_state)

In [ ]:
rollout_model = RolloutModel(pomdp, controller)

In [ ]:
# First timestep rollout...
rollout_tr = Gen.simulate(rollout_model, (0, PARAMS))

visualize_rollout(rollout_tr; goal=GOAL, title="First timestep")

In [ ]:
# Extend rollout to 40 steps...
rollout_tr = Gen.update(rollout_tr, (40, PARAMS))

visualize_rollout_path(rollout_tr; goal=GOAL, title="First timestep")

In [ ]:
visualize_rollout(rollout_tr; goal=GOAL, title="First timestep", fps=3)